In [61]:
sentences = [
    "오늘 날씨가 좋아서 나들이 가고 싶다.",
    "이 영화는 정말 재미있었어요.",
    "맛있는 음식을 먹으러 갈까요?",
    "운동을 하면 건강에 좋아지는 것 같아요.",
    "공부하기 싫어서 미루고 있어요.",
    "여행 계획을 세우고 있는데 어디로 갈까요?",
    "좋은 책을 읽으면 마음이 편안해져요.",
    "오늘은 친구들과 만나서 재미있게 놀았어요.",
    "새로운 언어를 배우는 것은 어려워도 흥미로워요.",
    "주말에 가족들과 함께 시간을 보내기로 했습니다."
]

### 자연어 전처리 

In [62]:
# 불용어 리스트 생성 (예시)
stopwords = ['가', '고', '을', '를', '이', '는']
from konlpy.tag import Okt

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# 불용어 리스트 생성 (예시)
stopwords = ['가', '고', '을', '를', '이', '는']

# 토크나이징 함수 정의
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [63]:
from mecab import MeCab
mecab = MeCab()

In [64]:
# 형태분석기 사용해 명사만 추출
# morphs_list = list()
# for sentence in sentences :
#     morphs = mecab.nouns(sentence)
#     morphs_list = morphs_list + morphs # 1차원
# morphs_list 

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer, use_idf=True)
features = vectorizer.fit_transform(sentences)
features.toarray()
# 디션너리가 만들어짐 

array([[0.46015789, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.46015789, 0.46015789, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.46015789, 0.        , 0.        ,
        0.        , 0.        , 0.39117625, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.51519219, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
      

### LDA 적용

In [66]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=3, random_state=111) #인스턴스화 #n_components 토픽의 갯수

In [67]:
lda_model.fit(features) #교육

LatentDirichletAllocation(n_components=3, random_state=111)

In [68]:
#lda_model.components_
#len(lda_model.components_[0])
topics_list = lda_model.components_[0] # 첫번째 토픽

### 토픽별 단어 표시 

In [69]:
dictionary_list= vectorizer.get_feature_names_out()
#len(vectorizer.get_feature_names_out())

In [70]:
# zip = 실제값과 토픽을 연결시켜줌 

list(zip(topics_list, dictionary_list))

[(0.33440990731900777, '가다'),
 (0.7992672461905107, '가족'),
 (0.33449535961542703, '갈다'),
 (0.8608367917667525, '건강'),
 (0.33438974543108546, '계획'),
 (0.9557349114059999, '공부'),
 (0.33440990731900777, '나들이'),
 (0.33440990731900777, '날씨'),
 (0.33452947443194514, '놀다'),
 (0.33498097590829673, '마음'),
 (0.33452947443194514, '만나다'),
 (0.334677720424076, '먹다'),
 (0.9557349114060001, '미루다'),
 (1.0362219540441318, '배우다'),
 (0.7992672461905106, '보내다'),
 (0.33438974543108546, '세우다'),
 (0.7992672461905106, '시간'),
 (0.33440990731900777, '싶다'),
 (0.33438974543108546, '어디'),
 (1.036221954044132, '언어'),
 (0.33438974543108546, '여행'),
 (0.3350113794266822, '영화'),
 (0.3344182379368015, '오늘'),
 (0.8608367917667525, '운동'),
 (0.334677720424076, '음식'),
 (0.33498097590829673, '읽다'),
 (0.3350113794266822, '정말'),
 (0.8608367917667525, '좋아지다'),
 (0.7992672461905106, '주말'),
 (0.33452947443194514, '친구'),
 (1.5390286363041017, '하다')]

In [71]:
import pandas as pd

In [72]:
# df_datas = [topics_list, dictionary_list]
# df_topics = pd.DataFrame(data=df_datas)
# df_topics = df_topics.T # 행열의 배열이 바뀜 가로 <-> 세로
# df_topics[:2]

In [73]:
# df_topics.columns

In [74]:
# df_topics.sort_values(0, ascending=False) #by=[0]

### 댓글과 주요 토픽 연결

In [75]:
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:4])# 시리즈 형식으로 출력 get values from series / index 
    print(topics_text)
    topics_list.append(topics_text)

하다 언어 배우다 미루다
오늘 영화 정말 놀다
갈다 읽다 마음 먹다


In [76]:
' '.join(df_topics[1].values[:4]) #시리즈를 하나에 문장 생성

'갈다 읽다 마음 먹다'

In [77]:
topics_list_add = [topics_list, ['Topic0', 'Topic1', 'Topic2']]
df_topics_keywords = pd.DataFrame(topics_list_add)
df_topics_keywords.T

,0,1
0,하다 언어 배우다 미루다,Topic0
1,오늘 영화 정말 놀다,Topic1
2,갈다 읽다 마음 먹다,Topic2


In [78]:
topics_output = lda_model.transform(features)
topics_output

array([[0.10462299, 0.7905512 , 0.10482582],
       [0.13946382, 0.72088285, 0.13965333],
       [0.12347941, 0.12362429, 0.7528963 ],
       [0.77339137, 0.11328861, 0.11332002],
       [0.75164634, 0.12416304, 0.12419063],
       [0.10458222, 0.10474923, 0.79066855],
       [0.13943843, 0.13959605, 0.72096551],
       [0.11268585, 0.77443415, 0.11288   ],
       [0.72035161, 0.13980797, 0.13984042],
       [0.78957076, 0.10519796, 0.10523128]])

## 각 댓글별 topic 분류 

In [79]:
lda_model.n_components

3

In [80]:
df_topics_score = pd.DataFrame(topics_output) # 각 댓글 당 토픽 분류 점수 

In [81]:
# 각 댓글 마다 토픽 분류 지정
import numpy as np
dominant_in_topic = np.argmax(topics_output, axis=1) # 댓글의 갯수만큼 분류 출력

In [82]:
df_topics_score['dominant_topic'] = dominant_in_topic
df_topics_score[:3]

# dominant_in_topic의 열 안에 가장 큰 값의 인덱스 

,0,1,2,dominant_topic
0,0.104623,0.790551,0.104826,1
1,0.139464,0.720883,0.139653,1
2,0.123479,0.123624,0.752896,2


In [84]:
df_topics_score['sentences'] = sentences
df_topics_score

,0,1,2,dominant_topic,sentences
0,0.104623,0.790551,0.104826,1,오늘 날씨가 좋아서 나들이 가고 싶다.
1,0.139464,0.720883,0.139653,1,이 영화는 정말 재미있었어요.
2,0.123479,0.123624,0.752896,2,맛있는 음식을 먹으러 갈까요?
3,0.773391,0.113289,0.113320,0,운동을 하면 건강에 좋아지는 것 같아요.
4,0.751646,0.124163,0.124191,0,공부하기 싫어서 미루고 있어요.
5,0.104582,0.104749,0.790669,2,여행 계획을 세우고 있는데 어디로 갈까요?
6,0.139438,0.139596,0.720966,2,좋은 책을 읽으면 마음이 편안해져요.
7,0.112686,0.774434,0.112880,1,오늘은 친구들과 만나서 재미있게 놀았어요.
8,0.720352,0.139808,0.139840,0,새로운 언어를 배우는 것은 어려워도 흥미로워요.
9,0.789571,0.105198,0.105231,0,주말에 가족들과 함께 시간을 보내기로 했습니다.


In [86]:
# 'dominant_topic' 열이 0인 행만 선택
filtered_df = df_topics_score[df_topics_score['dominant_topic'] == 0]
filtered_df

,0,1,2,dominant_topic,sentences
3,0.773391,0.113289,0.113320,0,운동을 하면 건강에 좋아지는 것 같아요.
4,0.751646,0.124163,0.124191,0,공부하기 싫어서 미루고 있어요.
8,0.720352,0.139808,0.139840,0,새로운 언어를 배우는 것은 어려워도 흥미로워요.
9,0.789571,0.105198,0.105231,0,주말에 가족들과 함께 시간을 보내기로 했습니다.
